#Introduction
##Prepare environment

In [1]:
# Cython needs to be installed before pycocotools
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib 

!pip install torchvision==0.5.0
#downgrade torch
!pip install torch==1.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 5.8 MB/s 
     |████████████████████████████████| 753.4 MB 6.6 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you h

In [2]:
!git clone https://github.com/dbolya/yolact.git 

Cloning into 'yolact'...
remote: Enumerating objects: 2936, done.
remote: Total 2936 (delta 0), reused 0 (delta 0), pack-reused 2936
Receiving objects: 100% (2936/2936), 21.20 MiB | 32.26 MiB/s, done.
Resolving deltas: 100% (2002/2002), done.


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


Sat Jun 25 14:04:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


Now adapt ```config.py``` file:
yolact/data/config.py - section dataset

I'm actually overwriting variables from the "base dataset"
 which is COCO[texto del enlace](https://)
```
# ----------------------- DATASETS ----------------------- #
...

xrays_dataset = dataset_base.copy({
  'name': 'Panoramic Xrays',
  'train_info': '/content/drive/MyDrive/TFM/dataset_for_prep/labels_json/all_train.json',
  'train_images': '/content/drive/MyDrive/TFM/dataset_for_prep/data',
  'valid_info': '/content/drive/MyDrive/TFM/dataset_for_prep/labels_json/all_val.json',
  'valid_images': '/content/drive/MyDrive/TFM/dataset_for_prep/data',
  'class_names': ('Tooth', #3
                  'Metal (Implant)', #4
                  'Metal', #6 
                  'Maxilla', #7
                  'Mandible', #8
                  'Inferior Alveolar Nerve', #9
                  'Maxillary Sinus', #12
                  'Plastic Restoration', #36
                  'Endodontic Treatment', #37
                  'Crown/Pontic', #38
                  'Metallic Restoration', #39
                  'Screw', #40
                  'Supernumerary'), #41
  'label_map': {3: 1,
                4: 2, 
                6: 3, 
                7: 4,
                8: 5,
                9: 6, 
                12: 7,
                36: 8,
                37: 9,
                38: 10,
                39: 11,
                40: 12, 
                41: 13}
})
```


Now on section YOLACT v1.0 CONFIGS 
```
# ----------------------- YOLACT v1.0 CONFIGS ----------------------- #
...

yolact_resnet50_xrays_config = yolact_resnet50_config.copy({
    'name': 'yolact_plus_resnet50_xrays',
    # Dataset stuff
    'dataset': xrays_dataset,
    'num_classes': len(xrays_dataset.class_names) + 1,

    # Image Size
    'max_size': 512,
})
```

now add  resnet50-19c8e357.pth to weights folder 

In [5]:
%cd /content/yolact/external/DCNv2

!python setup.py build develop

/content/yolact/external/DCNv2
running build
running build_ext
building '_ext' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/content
creating build/temp.linux-x86_64-3.7/content/yolact
creating build/temp.linux-x86_64-3.7/content/yolact/external
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2/src
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2/src/cpu
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2/src/cuda
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DWITH_CUDA -I/content/yolact/external/DCNv2/src -I/usr/local/lib/python3.7/dist-packages/torch/include -I/usr/local/lib/python3.7/dist-package

In [6]:
cd /content/yolact

/content/yolact


In [7]:
mkdir weights

In [8]:
cd /content/drive/MyDrive/TFM/yolaCT

/content/drive/MyDrive/TFM/yolaCT


In [9]:
import shutil

In [10]:
#shutil.copy('resnet50-19c8e357.pth', '/content/yolact/weights/')
#ROUND 1
#shutil.copy('yolact_plus_resnet50_xrays_5_273_interrupt.pth', '/content/yolact/weights/')
#ROUND 2
#shutil.copy('yolact_plus_resnet50_xrays_19_1015_interrupt.pth', '/content/yolact/weights/')
#ROUND 3
#shutil.copy('yolact_plus_resnet50_xrays_33_1752_interrupt.pth', '/content/yolact/weights/')
#ROUND 5
shutil.copy('yolact_plus_resnet50_xrays_83_4404_interrupt.pth', '/content/yolact/weights/')

'/content/yolact/weights/yolact_plus_resnet50_xrays_83_4404_interrupt.pth'

In [11]:
shutil.copy('config.py', '/content/yolact/data')

'/content/yolact/data/config.py'

In [12]:
cd /content/drive/MyDrive/TFM/dataset_for_prep


/content/drive/MyDrive/TFM/dataset_for_prep


In [13]:
shutil.copytree('data', '/content/images')

'/content/images'

# Train

In [ ]:
cd /content/yolact

/content/yolact


In [ ]:
!python ./train.py --config=yolact_resnet50_xrays_config --num_workers=0 --resume=weights/yolact_plus_resnet50_xrays_5_273_interrupt.pth  --batch_size=5
#!python ./train.py --config=yolact_resnet50_xrays_config --num_workers=0  --batch_size=5

Scaling parameters by 0.62 to account for a batch size of 5.
Per-GPU batch size is less than the recommended limit for batch norm. Disabling batch norm.
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Resuming training, loading weights/yolact_plus_resnet50_xrays_5_273_interrupt.pth...
Begin training!

/content/yolact/utils/augmentations.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
tcmalloc: large alloc 1121107968 bytes == 0x7d830000 @  0x7f754d3a3001 0x7f754a7ed1af 0x7f754a843c23 0x7f754a844a87 0x7f754a8e6823 0x5936cc 0x548c51 0x5127f1 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x4bcb19 0x532e76 0x59

In [ ]:
cd /content/yolact/weights

/content/yolact/weights


In [ ]:
shutil.copy('yolact_plus_resnet50_xrays_19_1015_interrupt.pth', '/content/drive/MyDrive/TFM/yolaCT')

'/content/drive/MyDrive/TFM/yolaCT/yolact_plus_resnet50_xrays_19_1015_interrupt.pth'

In [ ]:
cd /content/yolact

/content/yolact


In [ ]:
!python ./train.py --config=yolact_resnet50_xrays_config --num_workers=0 --resume=weights/yolact_plus_resnet50_xrays_19_1015_interrupt.pth  --batch_size=5

Scaling parameters by 0.62 to account for a batch size of 5.
Per-GPU batch size is less than the recommended limit for batch norm. Disabling batch norm.
loading annotations into memory...
Done (t=0.28s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Resuming training, loading weights/yolact_plus_resnet50_xrays_19_1015_interrupt.pth...
Begin training!

/content/yolact/utils/augmentations.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
tcmalloc: large alloc 1078337536 bytes == 0x7f1ca000 @  0x7f8ed4b3c001 0x7f8ed1f861af 0x7f8ed1fdcc23 0x7f8ed1fdda87 0x7f8ed207f823 0x5936cc 0x548c51 0x5127f1 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x4bcb19 0x532e76 0x

In [ ]:
cd /content/yolact/weights

/content/yolact/weights


In [ ]:
shutil.copy('yolact_plus_resnet50_xrays_33_1752_interrupt.pth', '/content/drive/MyDrive/TFM/yolaCT')

'/content/drive/MyDrive/TFM/yolaCT/yolact_plus_resnet50_xrays_33_1752_interrupt.pth'

## 4th round of training

In [ ]:
cd /content/yolact

/content/yolact


In [ ]:
!python ./train.py --config=yolact_resnet50_xrays_config --num_workers=0 --resume=weights/yolact_plus_resnet50_xrays_33_1752_interrupt.pth  --batch_size=5

Scaling parameters by 0.62 to account for a batch size of 5.
Per-GPU batch size is less than the recommended limit for batch norm. Disabling batch norm.
loading annotations into memory...
Done (t=0.62s)
creating index...
index created!
loading annotations into memory...
Done (t=0.25s)
creating index...
index created!
Resuming training, loading weights/yolact_plus_resnet50_xrays_33_1752_interrupt.pth...
Begin training!

/content/yolact/utils/augmentations.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
tcmalloc: large alloc 1871929344 bytes == 0xaf448000 @  0x7f5888882001 0x7f5885ccc1af 0x7f5885d22c23 0x7f5885d23a87 0x7f5885dc5823 0x5936cc 0x548c51 0x5127f1 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x4bcb19 0x532e76 0x

In [ ]:
cd /content/yolact/weights

/content/yolact/weights


In [ ]:
shutil.copy('yolact_plus_resnet50_xrays_40_2166_interrupt.pth', '/content/drive/MyDrive/TFM/yolaCT')

'/content/drive/MyDrive/TFM/yolaCT/yolact_plus_resnet50_xrays_40_2166_interrupt.pth'

In [ ]:
cd /content/yolact

/content/yolact


In [ ]:
!python ./train.py --config=yolact_resnet50_xrays_config --num_workers=0 --resume=weights/yolact_plus_resnet50_xrays_40_2166_interrupt.pth  --batch_size=5

Scaling parameters by 0.62 to account for a batch size of 5.
Per-GPU batch size is less than the recommended limit for batch norm. Disabling batch norm.
loading annotations into memory...
Done (t=0.37s)
creating index...
index created!
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
Resuming training, loading weights/yolact_plus_resnet50_xrays_40_2166_interrupt.pth...
Begin training!

/content/yolact/utils/augmentations.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
tcmalloc: large alloc 1189199872 bytes == 0xa3912000 @  0x7f4741ef7001 0x7f473f3411af 0x7f473f397c23 0x7f473f398a87 0x7f473f43a823 0x5936cc 0x548c51 0x5127f1 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x4bcb19 0x532e76 0x

In [ ]:
cd /content/yolact

/content/yolact


In [ ]:
!python ./train.py --config=yolact_resnet50_xrays_config --num_workers=0 --resume=weights/yolact_plus_resnet50_xrays_56_3006_interrupt.pth  --batch_size=5

Scaling parameters by 0.62 to account for a batch size of 5.
Per-GPU batch size is less than the recommended limit for batch norm. Disabling batch norm.
loading annotations into memory...
Done (t=0.27s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Resuming training, loading weights/yolact_plus_resnet50_xrays_56_3006_interrupt.pth...
Begin training!

/content/yolact/utils/augmentations.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
tcmalloc: large alloc 1881579520 bytes == 0xad852000 @  0x7fc7dba14001 0x7fc7d8e5e1af 0x7fc7d8eb4c23 0x7fc7d8eb5a87 0x7fc7d8f57823 0x5936cc 0x548c51 0x5127f1 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x4bcb19 0x532e76 0x

In [ ]:
cd /content/yolact/weights

/content/yolact/weights


In [ ]:
shutil.copy('yolact_plus_resnet50_xrays_73_3908_interrupt.pth', '/content/drive/MyDrive/TFM/yolaCT')

'/content/drive/MyDrive/TFM/yolaCT/yolact_plus_resnet50_xrays_73_3908_interrupt.pth'

In [ ]:
cd /content/yolact

/content/yolact


In [ ]:
!python ./train.py --config=yolact_resnet50_xrays_config --num_workers=0 --resume=weights/yolact_plus_resnet50_xrays_73_3908_interrupt.pth  --batch_size=5

Scaling parameters by 0.62 to account for a batch size of 5.
Per-GPU batch size is less than the recommended limit for batch norm. Disabling batch norm.
loading annotations into memory...
Done (t=0.28s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Resuming training, loading weights/yolact_plus_resnet50_xrays_73_3908_interrupt.pth...
Begin training!

tcmalloc: large alloc 2269200384 bytes == 0xb9640000 @  0x7f2539d60001 0x7f25371aa1af 0x7f2537200c23 0x7f2537201a87 0x7f25372a3823 0x5936cc 0x548c51 0x5127f1 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x4bcb19 0x532e76 0x594b72 0x548cc1 0x51566f 0x4bc98a 0x532e76 0x594b72 0x548cc1 0x51566f 0x549e0e 0x593fce 0x511e2c 0x4bc98a 0x533274 0x4d3969 0x512147 0x549e0e
/content/yolact/utils/augmentations.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) i

In [ ]:
cd /content/yolact/weights

/content/yolact/weights


In [ ]:
shutil.copy('yolact_plus_resnet50_xrays_83_4404_interrupt.pth', '/content/drive/MyDrive/TFM/yolaCT')

'/content/drive/MyDrive/TFM/yolaCT/yolact_plus_resnet50_xrays_83_4404_interrupt.pth'

In [14]:
cd /content/yolact

/content/yolact


In [15]:
!python ./train.py --config=yolact_resnet50_xrays_config --num_workers=0 --resume=weights/yolact_plus_resnet50_xrays_83_4404_interrupt.pth  --batch_size=5

Scaling parameters by 0.62 to account for a batch size of 5.
Per-GPU batch size is less than the recommended limit for batch norm. Disabling batch norm.
loading annotations into memory...
Done (t=0.69s)
creating index...
index created!
loading annotations into memory...
Done (t=0.31s)
creating index...
index created!
Resuming training, loading weights/yolact_plus_resnet50_xrays_83_4404_interrupt.pth...
Begin training!

/content/yolact/utils/augmentations.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
tcmalloc: large alloc 1208139776 bytes == 0xbe99e000 @  0x7ff478812001 0x7ff475c5c1af 0x7ff475cb2c23 0x7ff475cb3a87 0x7ff475d55823 0x5936cc 0x548c51 0x5127f1 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x4bcb19 0x532e76 0x

In [16]:
cd /content/yolact/weights

/content/yolact/weights


In [17]:
shutil.copy('yolact_plus_resnet50_xrays_100_5322_interrupt.pth', '/content/drive/MyDrive/TFM/yolaCT')

'/content/drive/MyDrive/TFM/yolaCT/yolact_plus_resnet50_xrays_100_5322_interrupt.pth'

# .

# Val and detect


In [19]:
cd /content/yolact

/content/yolact


In [ ]:
!python eval.py --trained_model=/content/yolact/weights/yolact_plus_resnet50_xrays_33_1752_interrupt.pth --config=yolact_resnet50_xrays_config --score_threshold=0.15 --top_k=50 

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Loading model... Done.

Processing Images  ██████████████████████████████     53 /     53 (100.00%)     2.27 fps        
Saving data...
Calculating mAP...

       |  all  |  .50  |  .55  |  .60  |  .65  |  .70  |  .75  |  .80  |  .85  |  .90  |  .95  |
-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
   box | 15.11 | 31.40 | 29.70 | 27.65 | 25.21 | 17.44 | 12.30 |  5.40 |  1.93 |  0.07 |  0.02 |
  mask |  8.68 | 19.16 | 18.08 | 15.59 | 13.57 | 10.82 |  6.91 |  2.32 |  0.37 |  0.03 |  0.00 |
-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+



In [20]:
cd /content/drive/MyDrive/TFM/yoloV5_data/all/images

/content/drive/MyDrive/TFM/yoloV5_data/all/images


In [21]:
shutil.copytree('det', '/content/detection_results/input')

'/content/detection_results/input'

In [22]:
cd /content/detection_results

/content/detection_results


In [23]:
mkdir output

In [24]:
cd /content/yolact

/content/yolact


In [25]:
!python eval.py --trained_model=/content/yolact/weights/yolact_plus_resnet50_xrays_100_5322_interrupt.pth --config=yolact_resnet50_xrays_config --score_threshold=0.5 --images=/content/detection_results/input:/content/detection_results/output --top_k=50
#!python eval.py --trained_model=/content/yolact/weights/yolact_plus_resnet50_xrays_33_1752_interrupt.pth --config=yolact_resnet50_xrays_config --score_threshold=0.2 --images=/content/detection_results/input:/content/detection_results/output --top_k=50

Loading model... Done.

/content/detection_results/input/94_copia.jpg -> /content/detection_results/output/94_copia.png
/content/detection_results/input/77_copia.jpg -> /content/detection_results/output/77_copia.png
/content/detection_results/input/69_copia.jpg -> /content/detection_results/output/69_copia.png
/content/detection_results/input/79_copia.jpg -> /content/detection_results/output/79_copia.png
/content/detection_results/input/98_copia.jpg -> /content/detection_results/output/98_copia.png
/content/detection_results/input/83_copia.jpg -> /content/detection_results/output/83_copia.png
/content/detection_results/input/89_copia.jpg -> /content/detection_results/output/89_copia.png
/content/detection_results/input/71_copia.jpg -> /content/detection_results/output/71_copia.png
/content/detection_results/input/88_copia.jpg -> /content/detection_results/output/88_copia.png
/content/detection_results/input/86_copia.jpg -> /content/detection_results/output/86_copia.png
/content/detecti

In [26]:
 !zip -r /content/detection_results.zip /content/detection_results

  adding: content/detection_results/ (stored 0%)
  adding: content/detection_results/input/ (stored 0%)
  adding: content/detection_results/input/94_copia.jpg (deflated 0%)
  adding: content/detection_results/input/77_copia.jpg (deflated 0%)
  adding: content/detection_results/input/69_copia.jpg (deflated 0%)
  adding: content/detection_results/input/79_copia.jpg (deflated 0%)
  adding: content/detection_results/input/98_copia.jpg (deflated 0%)
  adding: content/detection_results/input/83_copia.jpg (deflated 0%)
  adding: content/detection_results/input/89_copia.jpg (deflated 0%)
  adding: content/detection_results/input/71_copia.jpg (deflated 0%)
  adding: content/detection_results/input/88_copia.jpg (deflated 0%)
  adding: content/detection_results/input/86_copia.jpg (deflated 0%)
  adding: content/detection_results/input/85_copia.jpg (deflated 0%)
  adding: content/detection_results/input/73_copia.jpg (deflated 0%)
  adding: content/detection_results/input/74_copia.jpg (deflated 0%)

In [27]:
from google.colab import files
files.download("/content/detection_results.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>